Using [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) and langchain to test its performance.

In [23]:
import weaviate
import os

weaviate_client = weaviate.Client("http://localhost:8080",
    additional_headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY"),
    }
)

c:\anaconda\envs\dspy\Lib\site-packages\weaviate\warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.5.5.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [24]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core.response.notebook_utils import display_response
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

In [25]:
import requests

url = 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt'
r = requests.get(url)

file_path = 'example/paul_graham/paul_graham_essay.txt'
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, 'wb') as f:
    f.write(r.content)

In [26]:
documents = SimpleDirectoryReader("example/paul_graham/").load_data()

In [27]:
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI

## config llm like this
llm = OpenAI(model="gpt-4")

vector_store = WeaviateVectorStore(weaviate_client=weaviate_client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, index_name="test_prefix"
)

query_engine = index.as_query_engine(llm=llm, similarity_top_k=5) ## from this part check llm.
response = query_engine.query("What did the author do growing up?")

In [28]:
print(response)

The author spent his time outside of school primarily writing and programming. He wrote short stories and began programming on an IBM 1401 in 9th grade. He used an early version of Fortran to write programs on punch cards. Later, he started programming on a microcomputer, a TRS-80, where he wrote simple games, a program to predict the flight of his model rockets, and a word processor.


In [29]:
print(response.get_formatted_sources(length=200))

> Source (Doc id: fcdf80fc-2a74-4564-ab79-b4c23f9f9874): What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed...

> Source (Doc id: c0dc425d-d2d5-41fa-808d-b32625a9e052): I remember taking the boys to the coast on a sunny day in 2015 and figuring out how to deal with some problem involving continuations while I watched them play in the tide pools. It felt like I was...

> Source (Doc id: 3a239aa9-d4af-4872-85c3-56dbf8188004): If he even knew about the strange classes I was taking, he never said anything.

So now I was in a PhD program in computer science, yet planning to be an artist, yet also genuinely in love with Lis...

> Source (Doc id: 3d6b58fc-b3c0-42be-9fec-d33e8b63cbf7): I stuck it out for a few more months, then in desperation I went back to New York, where unless you understand about rent control you'll be surprised to hear I still 

With Reranker

In [30]:
rerank = FlagEmbeddingReranker(model="BAAI/bge-reranker-v2-m3", top_n=5)

In [31]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5, node_postprocessors=[rerank]) ## from this part check llm.
response = query_engine.query("What did the author do growing up?")

In [32]:
print(response)

The author spent his time outside of school primarily writing and programming. He wrote short stories and tried programming on the IBM 1401, a computer used by his school district. He later started programming more seriously when he got a TRS-80 microcomputer, creating simple games and a word processor among other things.


In [33]:
print(response.get_formatted_sources(length=200))

> Source (Doc id: e40a9c8b-1a25-42fc-b15d-7eb48a2389e1): But alas it was more like the Accademia than not. Better organized, certainly, and a lot more expensive, but it was now becoming clear that art school did not bear the same relationship to art that...

> Source (Doc id: c0dc425d-d2d5-41fa-808d-b32625a9e052): I remember taking the boys to the coast on a sunny day in 2015 and figuring out how to deal with some problem involving continuations while I watched them play in the tide pools. It felt like I was...

> Source (Doc id: fcdf80fc-2a74-4564-ab79-b4c23f9f9874): What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed...

> Source (Doc id: 3d6b58fc-b3c0-42be-9fec-d33e8b63cbf7): I stuck it out for a few more months, then in desperation I went back to New York, where unless you understand about rent control you'll be surprised to hear I still 